In [36]:
import pandas as pd
import plotly.express as px

In [37]:
# Cargar los DataFrames desde los archivos CSV
df_cali_hotels = pd.read_csv('california_hotels.csv')
df_cali_attributes = pd.read_csv('california_hotels_attributes.csv')

# Unificar horizontalmente usando pd.concat
df_combined = pd.concat([df_cali_hotels, df_cali_attributes], axis=1)

In [38]:
df_combined = df_combined.drop('Unnamed: 0', axis=1)

In [39]:
import pandas as pd
import json

# Reemplazar comillas simples por comillas dobles
df_combined['Scores'] = df_combined['Scores'].str.replace("'", "\"")

# Convertir las cadenas JSON a listas de diccionarios
df_combined['Scores'] = df_combined['Scores'].apply(lambda x: json.loads(x) if pd.notna(x) else [])

# Crear columnas para cada clave del diccionario
df_scores_expanded = pd.json_normalize(df_combined['Scores'])

# Concatenar las columnas de puntajes expandidas al DataFrame original
df_combined = pd.concat([df_combined, df_scores_expanded], axis=1)

In [40]:
# Convertir las últimas 8 columnas a cadenas
df_combined[df_combined.columns[-8:]] = df_combined[df_combined.columns[-8:]].astype(str)

In [41]:
# Eliminar las llaves y comillas simples de las últimas 8 columnas
for col in df_combined.columns[-8:]:
    df_combined[col] = df_combined[col].apply(lambda x: x.replace("{", "").replace("}", "").replace("'", "") if pd.notna(x) else None)


In [42]:
import pandas as pd

# Definimos una función para obtener el título de una celda
def get_title(cell_value):
    if pd.notna(cell_value):
        return cell_value.split(':')[0]
    else:
        return None

# Iteramos sobre las últimas 8 columnas y asignamos nuevos nombres
for col in df_combined.columns[-8:]:
    # Usamos apply para obtener el título de cada celda
    new_titles = df_combined[col].apply(get_title)
    
    # Asignamos los nuevos nombres a las columnas
    df_combined = df_combined.rename(columns=dict(zip([col]*len(new_titles), new_titles)))

# Ahora, df_combined tiene las columnas renombradas según los valores presentes en ellas



In [43]:
# Obtén el nombre de la anteúltima columna
anteultimate_col_index = -2
anteultimate_col_name = df_combined.columns[anteultimate_col_index]

# Copia el nombre de las columnas
new_columns = list(df_combined.columns)

# Cambia el nombre de la anteúltima columna a 'WiFi Gratis'
new_columns[anteultimate_col_index] = 'WiFi Gratis'

# Actualiza los nombres de las columnas en el DataFrame
df_combined.columns = new_columns


In [44]:
import pandas as pd

# Selecciona solo las últimas 8 columnas
last_8_columns = df_combined.iloc[:, -8:]

# Aplica una función para extraer y convertir los valores numéricos después de los dos puntos (:) en cada celda
numeric_values_only = last_8_columns.applymap(lambda x: pd.to_numeric(x.split(':')[-1], errors='coerce') if isinstance(x, str) and ':' in x else x)

# Actualiza las últimas 8 columnas en el DataFrame original con los valores numéricos
df_combined.iloc[:, -8:] = numeric_values_only


In [45]:
df_combined.head()

,state,city,name,avg_score,price,stars,reviews_url,Dirección,Descripción,Atributos,...,Longitud,Scores,Personal,Instalaciones y servicios,Limpieza,Confort,Relación calidad-precio,Ubicación,WiFi Gratis,None
0,California,Los Angeles,Central Los Angeles Hollywood Homes,88,418.541,4,https://www.booking.com/hotel/us/spacious-apar...,"3767 Greenwood Avenue, Los Ángeles, CA 90066, ...",Bright European Loft In Venice Beach está muy ...,NaN,...,-118.443532,[],None,None,None,None,None,None,None,None
1,California,Los Angeles,W Hollywood,73,849.752,4,https://www.booking.com/hotel/us/w-hollywood.e...,"2192 Beech Knoll Road, Hollywood, Los Ángeles,...",HOLLYWOOD HILLS JETLINER VIEWS at STUDIO 1080 ...,"['Terraza', 'Habitaciones sin humo', 'Calefacc...",...,-118.375404,"[{'Personal': '9.9'}, {'Instalaciones y servic...",9.9,9.8,9.8,9.8,9.7,9.9,9.0,None
2,California,Los Angeles,Park Plaza Lodge,86,466.927,3,https://www.booking.com/hotel/us/park-plaza-lo...,"13234 Rose Avenue, Venice Beach, Los Ángeles, ...","Venice beach house with pool, music studio est...","['Piscina al aire libre', 'Calefacción', 'Park...",...,-118.455345,"[{'Personal': '10'}, {'Instalaciones y servici...",10,10,10,10,9.6,9.6,None,None
3,California,Los Angeles,Moxy Downtown Los Angeles,79,904.364,4,https://www.booking.com/hotel/us/moxy-downtown...,"5801 Vesper Avenue, Los Ángeles, CA 91411, Est...",Gated private house with own parking Sherman O...,"['Terraza', 'Habitaciones sin humo', 'Calefacc...",...,-118.451436,"[{'Personal': '9.7'}, {'Instalaciones y servic...",9.7,9.7,10,9.7,9.7,9.7,None,None
4,California,Los Angeles,Hollywood Homes minutes to everything SPACIOUS...,88,463.104,4,https://www.booking.com/hotel/us/hollywood-cos...,"Pacific Beach, San Diego, 92109, Estados Unidos","Opal Oasis se encuentra en San Diego, en la re...","['Habitaciones sin humo', 'Parking en el aloja...",...,-117.257427,[],None,None,None,None,None,None,None,None


In [47]:
'''
import pandas as pd

# Tu DataFrame original (df)
# ...

# Columnas de scores
score_columns = ['Personal', 'Instalaciones y servicios', 'Limpieza', 'Confort', 'Relación calidad-precio', 'Ubicación', 'WiFi Gratis']

# Crear un nuevo DataFrame sin modificar el original
df_cleaned = df_combined.copy()

# Reemplaza 'None' con NaN en las columnas de scores
df_cleaned[score_columns] = df_cleaned[score_columns].replace('None', pd.NA)

# Elimina filas con valores no numéricos en las columnas de scores
df_cleaned[score_columns] = df_cleaned[score_columns].apply(pd.to_numeric, errors='coerce')
df_cleaned = df_cleaned.dropna(subset=score_columns)

# Agrega una columna 'state' al DataFrame limpio (df_cleaned) para fusionar con el DataFrame original
df_cleaned['state'] = df_combined['state']

# Calcula el promedio por estado para cada score
average_scores = df_cleaned.groupby('state')[score_columns].mean().reset_index()

# Fusiona el DataFrame original con el promedio de scores por estado
df_merged = pd.merge(df_combined, average_scores, on='state', suffixes=('_hotel', '_state'))

# Ahora, df_merged contiene las columnas originales y los promedios por estado
# Puedes usar df_merged para tu gráfico
'''


In [51]:
"""
import pandas as pd
import plotly.express as px

# Seleccionar las columnas relevantes
score_columns = [
    'Personal_hotel', 'Instalaciones y servicios_hotel', 'Limpieza_hotel',
    'Confort_hotel', 'Relación calidad-precio_hotel', 'Ubicación_hotel', 'WiFi Gratis_hotel'
]

# Configuración del gráfico
fig = px.bar(df_merged, x='state', y=score_columns,
             title='Scores del hotel comparados con el promedio del estado',
             labels={'value': 'Score', 'variable': 'Categoría'},
             template='plotly_white',
             hover_name='name')

# Mostrar el gráfico
fig.show()
"""
